In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys

In [2]:
query_url = 'https://csr.gov.in/CSR/search_page.php?query={}'

In [120]:
folder = 'CSR Year 2016-17/'

In [121]:
companies_invested_in_csr = pd.read_csv(folder + 'companies.csv')

In [122]:
companies_invested_in_csr.head()

,Unnamed: 0,name
0,0,22 Feet Tribal Worldwide Private Limited
1,1,A B T Limited
2,2,A G Industries Pvt Ltd
3,3,A G Superstructures Private Limited
4,4,A K Capital Services Limited


In [124]:
companies_invested_in_csr.shape

(2826, 2)

In [125]:
ctr = 0
CIN = []
print('Downloading...')
one_percent = companies_invested_in_csr.shape[0]//100
percent = 0
sys.stdout.write("\b"*len(str(percent)))
sys.stdout.write(str(percent))
sys.stdout.flush()
for company_name in companies_invested_in_csr['name']:
    query_page = requests.get(query_url.format(company_name.strip())).content
    soup = BeautifulSoup(query_page, 'html.parser')
    trs = soup.findAll('tr')
    for tr in trs:
        anchor = tr.a
        if anchor != None:
            CIN.append(anchor.text.strip())
            break
    else:
        CIN.append(None)
        
    ctr += 1
    if ctr % one_percent == 0:
        sys.stdout.write("\b"*len(str(percent)))
        percent += 1
        sys.stdout.write('#')
        sys.stdout.write(str(percent))
        sys.stdout.flush()

Downloading...
##########################26

SSLError: HTTPSConnectionPool(host='csr.gov.in', port=443): Max retries exceeded with url: /CSR/search_page.php?query=Encardio%20Rite%20Electronics%20Private%20Limited (Caused by SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')",),))

In [ ]:
len(CIN)

In [ ]:
companies_invested_in_csr['CIN'] = CIN

In [ ]:
companies_invested_in_csr.head()

none_indices = companies_invested_in_csr[pd.isna(companies_invested_in_csr['CIN'])].index
for i in none_indices:
    query_page = requests.get(query_url.format(companies_invested_in_csr.loc[i,'name'].strip())).content
    soup = BeautifulSoup(query_page, 'html.parser')
    trs = soup.findAll('tr')
    print(query_url.format(companies_invested_in_csr.loc[i,'name'].replace('  ', '%20').replace(' ', '%20').replace('\'','')))
    print(trs)
    for tr in trs:
        anchor = tr.a
        if anchor != None:
            print(anchor.text.strip())
            break
    else:
        #print(None)
        pass

In [ ]:
companies_in_graph_db_df = pd.read_csv('companies_in_graph_db.csv')

In [ ]:
tempDf = companies_in_graph_db_df.merge(companies_invested_in_csr, how='inner', on='CIN')

In [ ]:
tempDf.to_csv(folder + 'merged_companies.csv', columns=['CIN', 'name_x', 'name_y'])

In [ ]:
companies_invested_in_csr.to_csv(folder + 'companies.csv', columns=['name', 'CIN'])

In [ ]:
sum(pd.isna(companies_invested_in_csr['name']))